# Global Average Pooling

Global Average Pooling에 대한 소개는 [Network In Network](https://arxiv.org/pdf/1312.4400.pdf) 라는 페이퍼에서 소개가 됩니다.<br>
Network In Network의 논문을 정말 간단하게 말하면 기존 kernel/filter 를 일종의 generalized linear model로서 보는데, linearly separable한 것에 좋다고 설명합니다. 하지만 대부분의 데이터는 nonlinear manifold안에 존재하며 추상화를 높이기 위해서 MLPConv Layer를 추가합니다. (대충 짐작이 가듯이 MLP는 multi-layer perceptron을 가르킵니다.) 즉 receptive field를 연산후 output을 낼때 여기에 다시 MLP를 태워서 nonlinear function으로 만드는게 핵심입니다. 

하여튼.. 해당문서에서는 Network In Network가 중요한것이 아니고 바로 Global Average Pooling을 다뤄보겠습니다.<br>
참고한 문서는 [Network In Network](https://arxiv.org/pdf/1312.4400.pdf) 페이퍼안의 *3.2 Global Average Pooling* 을 참고하였습니다. <br>
(Network In Network는 제가 정말 시간이 날때.. 게임해야 되니 다루지 않겠습니다.) 

일반적으로 convolutional neural network 모델을 만들게 되면 output에서 fully-connected neural network를 태웁니다. <br>
이러한 방법은 convolution layers를 일종의 feature extractors로 보며, 뽑혀진 feature는 전통적인 방식으로 FC layer에서 classfication이 진행이 됩니다.

Fully-connected layer에는 다소 문제가 있습니다.<br>

1. Overfitting에 취약 -> 물론 Hinotn교수님에 의해서 제안된 Dropout을 사용해 overfitting을 줄일수 있습니다.
2. Positional data를 손실하게 됨 (Flatten하게 되면서 완벽한 손실. 물론 pooling에서도 손실은 여전히 있음) 
3. FC에서 느려진다는 말도 있는데. 이건 모델의 구조를 어떻게 했느냐에 따라서 달라지기 때문에 노 인정

GAP의 구현은 매우 쉽습니다. <br>
각각의 feature map을 분류하려는 category에 맞추면 됩니다. 예를 들어서 MNIST의 경우 10개의 categories가 있는데 이경우 feature map도 10개가 되면 됩니다.

예를 들어서 마지막 tensor의 shape (8, 8, 128) 이라면, 원래는 8 x 8 x 128로 flatten시켜야 합니다.<br>
이후 1개 이상의 fully-connected layers를 추가시키게 됩니다. (중간중간 dropout도 들어가고 BN도 들어가고.. ) <br>
layers타면서 dimension은 감소하게 되고 10개로 남기고 이후 softmax를 태웁니다. <br>
(페이퍼에서는 마지막 activation으로 softmax를 지칭했습니다.)

Global average pooling을 사용시 (8, 8, 10) tensor를 갖고 있다면, (8, 8)부분에 평균값을 구하고 결과값은 (1, 1, 10) tensor를 갖게 됩니다. 이후 해당 tensor를 1D vector로 reshape을 시키고 마지막으로 softmax activation을 태우게 됩니다. 이때 중간에 다른 레이어나 다른 연산이 들어가면 안됩니다. 그냥 이걸로 끝내는 것입니다.

즉 $ y = \text{softmax}(W * \text{flatten}(\text{GAP}(x) + b)) $  이게 아니고 
$ y = \text{softmax}(\text{flatten}(\text{GAP}(x))) $  이게 맞습니다.
